# Quantiles

https://machinelearningmastery.com/curve-fitting-with-python/

https://study.com/academy/lesson/interpolation-in-statistics-definition-formula-example.html

In [ ]:
import time
import pandas as pd

import numpy as np

from numpy import corrcoef as pcor

from numpy import exp as exp
from scipy import spatial

from itertools import combinations

import pickle

from datetime import datetime, timedelta

from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import numpy as np
def return_best_shapelet_pearson(vector):
    correlation_lst = []
    corrs = []
    for i in range(len(shapelet_standard_array)):
        score = similarity_metrix(shapelet_standard_array[i],vector)
        correlation_lst.append(shapelet_standard_names[i])
        corrs.append(score)
    scenario = corrs.index(max(corrs))
    return correlation_lst[scenario]

    
def return_all_shapelet_pearson(vector):
#     correlation_lst = []
    corrs = []
    for i in range(len(shapelet_standard_array)):
        score = similarity_metrix(shapelet_standard_array[i],vector)
#         correlation_lst.append(shapelet_standard_names[i])
        corrs.append(score)
    return corrs



def similarity_metrix(vector1,vector2):
    '''
    Here we have given user the flexibility to change the similarity function. Currently we have made it pearson correlation but it can be cosine
    
    1 - spatial.distance.cosine(vector1, vector2)
    '''
    similarity_value = round(pcor(vector1,vector2)[0][1],3)
    return similarity_value



In [364]:
export_visualizations = False ### when this is set to True, visualizations will be expored. If False, visualizations will only be displayed not exported
Data_refresh = 0 

# Smoothening_param= 3 ### make it dynamic in code below 


### if it is set to 1, data processing of all models needs to be done. As is this needs to be set1 when we feel model data need to be updated else 0


#### Ignore model List 
Ignore_model_list  = []

Ignore_State_list = ['Illinois', 'Arizona', 'Massachusetts',
       'Wisconsin', 'Texas', 'Nebraska', 'Utah', 'Oregon','United States','Washington',
       'New York', 'Rhode Island', 'Georgia', 'New Hampshire',
       'North Carolina', 'New Jersey', 'Colorado', 'Maryland', 'Nevada',
       'Tennessee', 'Hawaii', 'Indiana', 'Kentucky', 'Minnesota',
       'Oklahoma', 'Pennsylvania', 'South Carolina',
       'District of Columbia', 'Kansas', 'Missouri', 'Vermont',
       'Virginia', 'Connecticut', 'Iowa', 'Louisiana', 'Ohio', 'Michigan',
       'South Dakota', 'Arkansas', 'Delaware', 'Mississippi',
       'New Mexico', 'North Dakota', 'Wyoming', 'Alaska', 'Maine',
       'Alabama', 'Idaho', 'Montana', 'Puerto Rico', 'Virgin Islands',
       'Guam', 'West Virginia', 'Northern Mariana Islands',
       'American Samoa']

# select_state_list = ["Florida"]

# State_list = ["Florida"]


vector_length = (0,4)   ### 1 means using N-1 week value for defining shapelet and 4 weeks 4 weeks from future. 4 can't be changed because models generate only 4 weeks ahead predictions.
## (0,4) means look 4 weeks ahead in future while defining shapelet
## (1,4) means look 4 weeks ahead in future, 1 week in past basically N-1 week from actual covid incidence list for defining shapelet

history_weeks = vector_length[0]

future_weeks = vector_length[1]

assert future_weeks<=4,"Looking 4 weeks in future is fixed because our modelsgenerate 4 weeks ahead predictions. \n Please change vector_length[1]"


# Hyper Params 

Number_of_shapelets = 2 ### 6 

global Shapelet_length
Shapelet_length = vector_length[0]+vector_length[1]

shapelet_standard_array = [[0]*Shapelet_length for w in range(Number_of_shapelets)]
## here we have initialized an zero valued array of array.

#shapelet_standard_names = ["Flat","Inc",'Dec',"Surge",'Peaking',"Near Peak"]
shapelet_standard_names = ["Inc",'Dec']
assert len(shapelet_standard_names)==Number_of_shapelets, 'Size of array mismatch for shapelet_standard_names and value of  Number_of_shapelets'
# print('Size Mismacth')

### for this experiment, we have defined following shapelets 
# flat = [1.0, 1.00000001, 1.00000002, 1.00000003, 1.00000004]
# stable_inc = [1 ,2, 3, 4, 5]#, i.e., linear
# stable_dec = [5, 4, 3 ,2, 1]#, i.e., linear but decreasing
# surge =[exp(-1/2), exp(0/2), exp(1/2), exp(2/2), exp(3/2)]#, i.e., like exp(x)
# peaking = [-1*w for w in [exp(1/2) ,exp(0/2), exp(-1/2), exp(-2/2), exp(-3/2)]]#, i.e., like -exp(-x)
# at_near_peak=  [-1*w for w in [exp(-1/2) ,exp(0/2) ,exp(1/2) ,exp(2/2), exp(3/2)]]#, i.e., like -exp(x)

# shapelet_standard_array[0] = [1.0, 0, 1.0, 0.0000,1]
shapelet_standard_array[0] = [1 ,2, 3, 4]
shapelet_standard_array[1] = [5, 4, 3 ,2]
# shapelet_standard_array[3] = [1,2,4,8,16]
# shapelet_standard_array[4] = [-1*w for w in [exp(1/2) ,exp(0/2), exp(-1/2), exp(-2/2),exp(-3/2)]]
# shapelet_standard_array[5] = [-1*w for w in [exp(-1/2) ,exp(0/2) ,exp(1/2) ,exp(2/2),exp(3/2)]]

assert len(shapelet_standard_array[0])==Shapelet_length, 'Size of defined shapelet array mismatch for shapelet_standard_names and value of  Shapelet_length.please check vector_length'



In [ ]:
import numpy as np
import pandas as pd
import random
import glob

def valid_week_nbr(a_string):

    if [int(word) for word in a_string.split() if word.isdigit()][0]<5:
        return 1
    else:
        return 0
def interpolate(test_array):
    inter_list= []
    array_x = [w[0] for w in test_array]
    array_y = [w[1] for w in test_array]
#     sum_iter = 0
    for i in range(50):
        value_x = np.random.uniform(low=0.0, high=1.0)
        y_inter = np.interp(value_x,array_x,array_y)
        inter_list.append(round(y_inter,2))
    return inter_list


In [370]:
Source_path = "../Data_Sources/quantile_preprocessing/"

In [371]:
locatoins = pd.read_csv(Source_path+"locations.csv")

In [416]:
location_mapping = locatoins[['location','location_name']].set_index("location").to_dict()['location_name']

In [553]:
model_name = 'CEID-Walk'

runtype = "case"

valid_vals = ['US',	 '01',	 '02',	 '04',	 '05',	 '06',	 '08',	 '09',	 '10',	 '11',	 '12',	 '13',	 '15',	 '16',	 '17',	 '18',	 '19',	 '20',	 '21',	 '22',	 '23',	 '24',	 '25',	 '26',	 '27',	 '28',	 '29',	 '30',	 '31',	 '32',	 '33',	 '34',	 '35',	 '36',	 '37',	 '38',	 '39',	 '40',	 '41',	 '42',	 '44',	 '45',	 '46',	 '47',	 '48',	 '49',	 '50',	 '51',	 '53',	 '54',	 '55',	 '56',	 '60',	 '66',	 '69',	 '72',	 '74',	 '78']


In [42]:
## Load source files

In [458]:
path = Source_path+"2020-08-09-CEID-Walk.csv"

In [46]:
test_file = pd.read_csv(Source_path+"2020-08-09-CEID-Walk.csv")

In [508]:
path = "/Users/satwant/Downloads/CEID-Walk/"

In [509]:
filenames = glob.glob(path+"2*.csv") 

In [514]:
master_data = pd.DataFrame()
for i in filenames:
    tmp = pd.read_csv(i)
    ## filter for runtype
    cases_data_tmp = tmp[tmp['target'].str.contains(runtype)]

    ## filter for quantiles only 

    cases_data_tmp1 = cases_data_tmp[cases_data_tmp['type']=='quantile']

    cases_data = cases_data_tmp1[cases_data_tmp1['location'].isin(valid_vals)]

    cases_data['location'] = cases_data['location'].apply(lambda x : location_mapping.get(x,x)) 

    cases_data['valid_row'] = cases_data['target'].apply(valid_week_nbr)

    cases_data = cases_data[cases_data['valid_row']==1]
    
    master_data = pd.concat([master_data,cases_data],axis=0)
    
    

<ipython-input-514-86d49c88e514>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases_data['location'] = cases_data['location'].apply(lambda x : location_mapping.get(x,x))
<ipython-input-514-86d49c88e514>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases_data['valid_row'] = cases_data['target'].apply(valid_week_nbr)


In [518]:
cases_data = master_data.copy()

In [519]:
cases_data['ordered_pair'] =cases_data[['quantile', 'value']].apply(tuple, axis=1)



In [520]:
cases_subset = cases_data[['forecast_date','location','target','ordered_pair','target_end_date']]


cases_rolledup = cases_subset.groupby(['forecast_date','location','target','target_end_date'])['ordered_pair'].apply(list).reset_index()



In [521]:
pd.set_option('display.max_columns', None)


In [524]:
cases_rolledup['IR_value_50sim'] = cases_rolledup['ordered_pair'].apply(interpolate)

In [525]:
cases_rolledup['week_interpolated_array'] =cases_rolledup[['target', 'IR_value_50sim']].apply(tuple, axis=1)


In [526]:
cases_rolledup1 = cases_rolledup.groupby(['forecast_date','location'])['week_interpolated_array'].apply(list).reset_index()


In [538]:
def return_Similarity_score_all(x):
    try:
    #     print(x[0][0])
    #     frst_week = []
        for item in x:
            if item[0].find("1")>-1:
                frst_week=item[1]
                break
        second_week = []
        for item in x:
            if item[0].find("2")>-1:
                second_week=item[1]
                break
        thrd_week =[]
        for item in x:
            if item[0].find("3")>-1:
                thrd_week=item[1]
                break
        frth_week=[]
        for item in x:
            if item[0].find("4")>-1:
                frth_week=item[1]
                break
        interpolated_array = []


        for i in range(len(frst_week)):
            vector = [frst_week[i],second_week[i],thrd_week[i],frth_week[i]]
    #         print(vector)
            interpolated_array.append(return_all_shapelet_pearson(vector))


        return interpolated_array
    except:
        return "NAN"
    
def aggregated_similarity(data):
    try:
        a = np.array(data)

        aggs = a.mean(axis=0)

        xx = [round(w,4) for w in aggs]
        return xx
    except:
        print(data)
    
from datetime import date


d0 = date(2020, 1, 22)

def format_dt_int(w):
    wsplit = w.split("-")
    yr = int(wsplit[0])
    mth=int(wsplit[1])
    dt = int(wsplit[2])
    d1 = date(yr, mth, dt)
    delta = d1 - d0
    return delta.days

import math
def is_valid(x):
    for i in x:
        if math.isnan(i):
            return 0
    return 1             


In [539]:

cases_rolledup1['All_iterpol_similar'] = cases_rolledup1['week_interpolated_array'].apply(return_Similarity_score_all)


In [541]:
cases_rolledup1 = cases_rolledup1[cases_rolledup1['All_iterpol_similar']!='NAN']

In [543]:

cases_rolledup1['aggregated_similarity'] = cases_rolledup1['All_iterpol_similar'].apply(aggregated_similarity)


<ipython-input-543-84d599ac45d6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases_rolledup1['aggregated_similarity'] = cases_rolledup1['All_iterpol_similar'].apply(aggregated_similarity)


In [546]:
cases_rolledup1['forecast_date1'] = cases_rolledup1['forecast_date'].apply(format_dt_int)


<ipython-input-546-ef84189f3f49>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases_rolledup1['forecast_date1'] = cases_rolledup1['forecast_date'].apply(format_dt_int)


In [547]:
cases_rolledup1['model_name'] = model_name

<ipython-input-547-e3a45e6bf7ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases_rolledup1['model_name'] = model_name


In [548]:
cases_rolledup1_subset = cases_rolledup1[['model_name','forecast_date1','location','aggregated_similarity']]

In [551]:
cases_rolledup1_subset['valid_row'] = cases_rolledup1_subset['aggregated_similarity'].apply(is_valid)

cases_rolledup1_subset = cases_rolledup1_subset[cases_rolledup1_subset["valid_row"]==1]

<ipython-input-551-67fb048b22af>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases_rolledup1_subset['valid_row'] = cases_rolledup1_subset['aggregated_similarity'].apply(is_valid)


In [552]:
cases_rolledup1_subset

,model_name,forecast_date1,location,aggregated_similarity,valid_row
0,CEID-Walk,200,Alabama,"[-0.1308, 0.1308]",1
1,CEID-Walk,200,Alaska,"[0.0851, -0.0851]",1
3,CEID-Walk,200,Arizona,"[0.0404, -0.0404]",1
4,CEID-Walk,200,Arkansas,"[0.0632, -0.0632]",1
5,CEID-Walk,200,California,"[0.343, -0.343]",1
...,...,...,...,...,...
3757,CEID-Walk,838,Virginia,"[-0.0418, 0.0418]",1
3758,CEID-Walk,838,Washington,"[0.0081, -0.0081]",1
3759,CEID-Walk,838,West Virginia,"[-0.0595, 0.0595]",1
3760,CEID-Walk,838,Wisconsin,"[0.1043, -0.1043]",1


In [492]:
# Shapelet_ensemble = [round(w,4) for w in \
#                      list(np.array(cases_rolledup1_subset['aggregated_similarity'].to_list()).mean(axis=0))]
                     
                     

In [493]:
# Shapelet_ensemble

In [263]:
processed_df = cases_rolledup.pivot(index=['forecast_date','location'], columns='target_end_date', values='IR_value_50sim').reset_index()



In [265]:
current_date = processed_df.forecast_date.unique()[0]

In [266]:
processed_df.replace(current_date,"NaN",inplace=True)

In [267]:
original_columns = list(processed_df.columns)

In [271]:
processed_df.columns = [current_date]+[original_columns[1]]+original_columns[2:]

In [273]:
processed_df_final = processed_df[[original_columns[1]]+[current_date]+original_columns[2:]]

In [274]:
processed_df_final

,location,2020-08-09,2020-08-15,2020-08-22,2020-08-29,2020-09-05
0,01,NaN,10063.472173,10067.909694,9931.869003,9827.519538
1,01001,NaN,96.476641,104.130352,107.947853,115.464909
2,01003,NaN,484.352074,416.586318,447.520027,468.479518
3,01005,NaN,26.453318,17.391378,15.118307,21.016704
4,01007,NaN,63.969214,65.559894,65.205716,63.618655
...,...,...,...,...,...,...
3193,66,NaN,57.058448,54.910670,50.342924,51.444087
3194,69,NaN,4.185623,3.814157,4.110112,4.133898
3195,72,NaN,3542.295718,3457.178047,3553.352069,3502.905559
3196,78,NaN,111.090125,123.443248,97.755159,129.874003


## Next Steps
1) Location column mapping and filters<br><br>  locations.csv in data folder
2) Aggregation criterion for multiple iterations of interpolation<br><br>
3) should we keep same code by just adding a hyper param which runs the
interpolation preprocessing script and changes visualization titles to (Quantile)?<br><br>
4) From Washington, California, Florida, 2 clusters pretty evident . so need to decide if we want to reduce # of shapes<br><br>
5) Research paper next steps as deadlines approaching<br><br>

In [277]:
import datetime
from datetime import datetime, timedelta


In [281]:
date_columns = list(processed_df_final.columns)[1:]

In [283]:
# N = 200
# t = '2020-01-22'
# format = '%Y-%m-%d'
# now = datetime.strptime(t,format)
# after = now + timedelta(days = N)
# print(now)
# print(after)

2020-01-22 00:00:00
2020-08-09 00:00:00


In [289]:
from datetime import date

new_columns = []

d0 = date(2020, 1, 22)

for w in date_columns:
    wsplit = w.split("-")
    yr = int(wsplit[0])
    mth=int(wsplit[1])
    dt = int(wsplit[2])
    d1 = date(yr, mth, dt)
    delta = d1 - d0
    print(delta.days)
    new_columns.append(delta.days)

200
206
213
220
227


In [293]:
processed_df_final.columns = [list(processed_df_final.columns)[0]]+new_columns

In [294]:
processed_df_final.head()

,location,200,206,213,220,227
0,01,NaN,10063.472173,10067.909694,9931.869003,9827.519538
1,01001,NaN,96.476641,104.130352,107.947853,115.464909
2,01003,NaN,484.352074,416.586318,447.520027,468.479518
3,01005,NaN,26.453318,17.391378,15.118307,21.016704
4,01007,NaN,63.969214,65.559894,65.205716,63.618655
